<a href="https://colab.research.google.com/github/amara929/amara929/blob/main/Advanced_CNN_architectures_ResNet%2C_Inception_and_DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#ResNet block, Inception module, and DenseNet block functions
#(Same as in the previous example)

...

Ellipsis

The provided code snippet imports TensorFlow and Keras, which are widely used libraries for deep learning. It also imports the layers module from Keras, which provides building blocks for constructing neural networks. The comment suggests that the script includes functions for implementing a ResNet block, an Inception module, and a DenseNet block, which are key components in modern convolutional neural networks (CNNs). These architectures are designed to improve feature extraction and gradient flow in deep networks: ResNet introduces residual connections to address vanishing gradients, Inception combines multiple convolutional filters for multi-scale feature extraction, and DenseNet connects each layer to every subsequent layer to enhance feature reuse. However, the actual implementation of these modules is missing, as indicated by the ellipsis (...).









In [ ]:
#ResNet block
def resnet_block(inputs, filters,kernel_size=3, strides=1, conv_shortcut=False):
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)

    if conv_shortcut:
        shortcut = layers.Conv2D(filters, 1, strides=strides)(inputs)
        shortcut = layers.BatchNormalization()(shortcut)
    else:
        shortcut = inputs
    x = layers.add([x, shortcut])
    x = layers.ReLU()(x)
    return x


This code defines a ResNet (Residual Network) block, which is a fundamental building block in deep residual learning. The function resnet_block takes an input tensor (inputs) and applies two convolutional layers with batch normalization and ReLU activation. If conv_shortcut is True, a convolutional shortcut (1x1 convolution) is applied to match the dimensions of the input with the output, ensuring compatibility when the number of filters or stride changes. Otherwise, the input is passed directly as a shortcut connection. The output of the two convolutional layers is then added to the shortcut connection, forming a residual connection, which helps mitigate the vanishing gradient problem and allows deeper networks to be trained effectively. Finally, a ReLU activation is applied to the result before returning the transformed tensor.

In [ ]:
#Inception module
def inception_module(inputs, filters):
    conv1x1 = layers.Conv2D(filters, 1, padding='same', activation='relu')(inputs)
    conv3x3 = layers.Conv2D(filters, 3, padding='same', activation='relu')(inputs)
    conv5x5 = layers.Conv2D(filters, 5, padding='same', activation='relu')(inputs)
    pool = layers.MaxPooling2D(3, strides=1, padding='same')(inputs)
    pool = layers.Conv2D(filters, 1, padding='same', activation='relu')(pool)
    output = layers.concatenate([conv1x1, conv3x3, conv5x5, pool], axis=-1)
    return output


This code defines an Inception module, a key component of the Inception network (GoogLeNet), which enhances feature extraction by using multiple convolutional filters of different sizes in parallel. The function inception_module takes an input tensor and applies four different operations: (1) a 1×1 convolution for dimensionality reduction and feature extraction, (2) a 3×3 convolution for capturing medium-sized spatial patterns, (3) a 5×5 convolution for capturing larger spatial features, and (4) a 3×3 max pooling layer, followed by a 1×1 convolution to reduce dimensionality. These feature maps are then concatenated along the channel axis to form the final output, allowing the network to learn multi-scale features efficiently. This module helps improve performance by enabling the model to extract fine, medium, and coarse details simultaneously while keeping computational costs manageable.

In [ ]:
#DenseNet block
def densenet_block(inputs, growth_rate):
    x = layers.BatchNormalization()(inputs)
    x = layers.ReLU()(x)
    x = layers.Conv2D(growth_rate, 3, padding='same')(x)
    x = layers.concatenate([inputs, x], axis=-1)
    return x

This code defines a DenseNet block, a core component of DenseNet (Densely Connected Convolutional Networks), which enhances feature reuse and gradient flow. The function densenet_block takes an input tensor and first applies batch normalization and ReLU activation to standardize and introduce non-linearity. It then applies a 3×3 convolution with growth_rate filters, which determines the number of new feature maps learned. The output of this convolution is then concatenated with the original input along the channel axis, ensuring that all previous feature maps are preserved and passed to subsequent layers. This dense connectivity helps reduce the number of parameters, improve gradient propagation, and encourage feature reuse, making DenseNet highly efficient compared to traditional deep networks.

In [ ]:
#Build the model
def build_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    #ResNet-like architecture
    x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    x = resnet_block(x, 64)
    x = resnet_block(x, 64)
    x = resnet_block(x, 128, strides=2, conv_shortcut=True)
    x = resnet_block(x, 128)

    #Inception module
    x = inception_module(x, 128)

    #DenseNet block
    x = densenet_block(x, 32)
    x = densenet_block(x, 32)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, outputs)
    return model


This code defines the build_model function, which constructs a hybrid deep learning model by integrating elements from ResNet, Inception, and DenseNet architectures. The model starts with an initial convolutional layer (7×7, 64 filters, stride 2), followed by batch normalization, ReLU activation, and max pooling, which help downsample and normalize the input. Next, it incorporates ResNet blocks, which include residual connections to improve gradient flow and train deeper networks. After two 64-filter ResNet blocks, it adds a 128-filter ResNet block with a convolutional shortcut (stride 2) for downsampling, followed by another 128-filter ResNet block. The model then applies an Inception module, which extracts multi-scale features using different filter sizes. Two DenseNet blocks follow, where each layer concatenates its output with previous feature maps to improve feature reuse. Finally, global average pooling reduces the feature maps to a vector, which is passed to a fully connected softmax layer for classification. The function returns a Keras model that balances feature extraction, depth, and computational efficiency.










In [ ]:
#Load and preprocess the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


This code loads and preprocesses the CIFAR-10 dataset, a standard dataset for image classification consisting of 60,000 color images (32×32 pixels) in 10 classes. The dataset is split into training (50,000 images) and testing (10,000 images) sets using keras.datasets.cifar10.load_data(). The pixel values of the images (x_train and x_test) are then normalized by converting them to float32 and scaling them to the range [0, 1] (by dividing by 255.0), which helps improve training stability. The labels (y_train and y_test), initially in integer form, are one-hot encoded using keras.utils.to_categorical(y, 10), transforming them into binary vectors of length 10 (one for each class). This preprocessing step ensures that the data is in a suitable format for training a neural network.










In [ ]:
#Build and compile the model
input_shape =  (32, 32, 3)
num_classes = 10
model = build_model(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

This code builds and compiles a deep learning model for classifying images from the CIFAR-10 dataset. It first defines the input shape as (32, 32, 3), representing 32×32 pixel RGB images, and sets the number of classes to 10 (since CIFAR-10 has 10 categories). The build_model function is then called to construct the model using a combination of ResNet, Inception, and DenseNet blocks. After building the model, it is compiled with the Adam optimizer, which is an adaptive learning rate optimization algorithm that helps speed up convergence. The categorical cross-entropy loss function is used because the task is multi-class classification, and accuracy is specified as the evaluation metric. This compilation step prepares the model for training by defining how it will update weights and measure performance.

In [ ]:
#Train the model
batch_size = 128
epochs = 10
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - accuracy: 0.4238 - loss: 1.6546 - val_accuracy: 0.2918 - val_loss: 2.0843
Epoch 2/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.6343 - loss: 1.0149 - val_accuracy: 0.5936 - val_loss: 1.1312
Epoch 3/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.7178 - loss: 0.8017 - val_accuracy: 0.6424 - val_loss: 1.0322
Epoch 4/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.7698 - loss: 0.6481 - val_accuracy: 0.5360 - val_loss: 1.5100
Epoch 5/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8053 - loss: 0.5530 - val_accuracy: 0.6848 - val_loss: 0.9542
Epoch 6/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.8391 - loss: 0.4561 - val_accuracy: 0.6368 - val_loss: 1.2269
Epoch 7/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.8690 - loss: 0.3692 - val_accuracy: 0.5942 - val_loss: 1.4952
Epoch 8/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8944 - loss: 0.3047 - val_acc

This code trains the deep learning model on the CIFAR-10 dataset using the model.fit function. It sets the batch size to 128, meaning the model processes 128 images at a time before updating its weights. The number of epochs is set to 10, meaning the model will go through the entire training dataset 10 times. The training data (x_train and y_train) is used, with 10% of it reserved for validation (validation_split=0.1), allowing the model to evaluate its performance on unseen data during training. The fit function optimizes the model's parameters using the Adam optimizer, minimizing the categorical cross-entropy loss, while tracking performance with accuracy. This step is essential for enabling the model to learn patterns in the data and improve its classification ability.

In [ ]:
#Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.7297 - loss: 1.1086
Test accuracy: 0.7263000011444092


This code evaluates the trained model on the test dataset (x_test and y_test) using the model.evaluate function. It calculates the test loss and test accuracy, which measure how well the model generalizes to new, unseen data. The test_loss represents the value of the loss function (categorical cross-entropy in this case) on the test set, while test_acc represents the percentage of correct predictions made by the model. After evaluating the model, it prints the test accuracy, providing an indication of the model's performance in classifying the CIFAR-10 test images. This step is crucial for assessing how well the model performs outside of the training data.